##Check GPU 




In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=f3c07cf3f6b6e8c86dd9d23f9985b0e4b3995a487335a79c24e625a37bf7dd53
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 111.6 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [2]:
cd ..


/


In [3]:
!nvidia-smi

Thu Aug 20 11:31:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
!kill process_id

/bin/bash: line 0: kill: process_id: arguments must be process or job IDs


In [5]:
ls


bin/      dev/   lib32/  opt/   sbin/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/  srv/    tmp/
content/  home/  media/  root/  swift/  tools/
datalab/  lib/   mnt/    run/   sys/    usr/


##Loading the Data

In [6]:
import numpy as np
from torch.utils.data import Dataset
import torch 
from torch import nn
from torch import optim
import copy 
import matplotlib.pyplot as plt
from torch.autograd import Variable 
import numpy as np
 
data = np.load('/content/drive/My Drive/data.gz').reshape(-1,700,57)
train,label = np.split(data,[22],axis=2)
label = label[:,:,0:8]
ss_label = label[:,:,0:13]

train, test = np.split(train,[5000],axis=0)
test, val = np.split(test,[267],0)

label, test_label = np.split(label,[5000],axis=0)
test_label, val_label = np.split(test_label,[267],0)

ss_label, ss_test_label = np.split(ss_label,[5000],axis=0)
ss_test_label, ss_val_label = np.split(ss_test_label,[267],0)
#train1, train2 = np.split(train,[350],axis=1)
#label1, label2 = np.split(label,[350],axis=1)

#label1 = torch.tensor(label1,dtype=torch.int64)
#label2 = torch.tensor(label2,dtype=torch.int64)

#train1 = torch.tensor(train1,dtype=torch.float64)
#train2 = torch.tensor(train2,dtype=torch.float64)

#print(train1.shape,label1.shape)


#print(label[0,0:10,:])


In [7]:
p

NameError: ignored

##Generate the Three State Labels

In [ ]:
#Generate the three state labels
label_3 = np.roll(label,-1,2)
test_label_3 = np.roll(test_label,-1,2)
val_label_3 = np.roll(val_label,-1,2)


def transfer_to_three(eight, num_seq):
    #e1, e2, e3 = np.split(eight,axis=2,indices_or_sections=np.array([2,5]))
    #for n in [e1,e2,e3]:
    #    n = n.tolist()
    eight = eight.tolist()
    for i in range(num_seq):
        for j in range(700):
           if 1 in eight[i][j][0:2]:
              eight[i][j] = [1,0,0]
           elif 1 in eight[i][j][2:5]:
              eight[i][j] = [0,1,0]
           elif 1 in eight[i][j][5:8]:
              eight[i][j] = [0,0,1] 
           else:
              eight[i][j] = [0,0,0]
    
    #e1 = torch.tensor(e1,dtype=torch.int64)
    #e2 = torch.tensor(e2,dtype=torch.int64)
    #e3 = torch.tensor(e3,dtype=torch.int64)
    #eight = torch.cat((e1,e2,e3),dim = 1)
    return eight

test_label_3 = torch.tensor(transfer_to_three(test_label_3,267),dtype=torch.int64)
label_3 = torch.tensor(transfer_to_three(label_3,5000),dtype=torch.int64)
val_label_3 = torch.tensor(transfer_to_three(val_label_3,267),dtype=torch.int64)

In [ ]:
label = torch.tensor(label,dtype=torch.int64)
train = torch.tensor(train,dtype=torch.float64)
test_label = torch.tensor(test_label,dtype=torch.int64)
val = torch.tensor(val,dtype=torch.float64)
val_label = torch.tensor(val_label,dtype=torch.int64)
test = torch.tensor(test,dtype=torch.float64)
ss_label = torch.tensor(ss_label,dtype=torch.int64)
ss_test_label = torch.tensor(ss_test_label,dtype=torch.int64)
ss_val_label = torch.tensor(ss_val_label,dtype=torch.int64)




In [ ]:

label_dis = torch.argmax(label,dim=2)
print(label_dis.shape)

##Attention Mechisam


In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


def attention(q, k, v, d_k, dropout=None):
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    scores = F.softmax(scores, dim=-1)
        
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        #self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v):
        
        # perform linear operation and split into h heads
 
        k = self.k_linear(k).view(-1, self.h, self.d_k)
        q = self.q_linear(q).view(-1, self.h, self.d_k)
        v = self.v_linear(v).view(-1, self.h, self.d_k)
    
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(0,1)
        q = q.transpose(0,1)
        v = v.transpose(0,1)
       
# calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(0,1).contiguous()\
        .view(-1, self.d_model)
        output = self.out(concat)
   
        return output
    
    
    
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__() 
        self.linear_1 = nn.Linear(d_model, d_ff)
        #self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = F.relu(self.linear_1(x))
        x = self.linear_2(x)
        return x
   ############################################################################################################ 
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=700):
        self.d_model = d_model 
        super(PositionalEncoding, self).__init__()
        #self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model).to('cuda')
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model%2 == 0:
            pe[:, 1::2] = torch.cos(position * div_term)
        else:
            pe[:, 1:10:2] = torch.cos(position * (div_term[0:1]))
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

    #######################################################################################################
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
     
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm
    

## Model Backbone


In [ ]:
# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads,d_ff):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model,d_ff)
        #self.dropout_1 = nn.Dropout(dropout)
        #self.dropout_2 = nn.Dropout(dropout)
            
    def forward(self, x):
        x2 = self.norm_1(x)
        x = x + self.attn(x2,x2,x2)
        x2 = self.norm_2(x)
        x = x + self.ff(x2)
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads,d_ff):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        #self.dropout_1 = nn.Dropout(dropout)
        #self.dropout_2 = nn.Dropout(dropout)
        #self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model,d_ff)
       
        
    def forward(self, x, e_outputs):
       
        x2 = self.norm_1(x)
        x = x + self.attn_1(x2, x2, x2)
        x2 = self.norm_2(x)
        x = x + self.attn_2(x2, e_outputs, e_outputs)
        x2 = self.norm_3(x)
        x = x + self.ff(x2)
        return x
# We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


class Encoder(nn.Module):
    def __init__(self, src_emb, N, heads,label_dim,d_ff):
        super().__init__()
        self.N = N
        self.pe = PositionalEncoding(22)
        self.layers = get_clones(EncoderLayer(src_emb, heads,d_ff), N)
        self.norm = Norm(src_emb)
        self.dimension_unify = nn.Linear(22,label_dim)
        
    def forward(self, x):
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x)
        x = self.norm(x)
        return self.dimension_unify(x)
    
class Decoder(nn.Module):
    def __init__(self, trg_emb, N, heads,d_ff,label_dim):
        super().__init__()
        self.N = N
        self.pe = PositionalEncoding(label_dim)
        self.layers = get_clones(DecoderLayer(trg_emb, heads,d_ff), N)
        self.norm = Norm(trg_emb)
        
    def forward(self, x, e_outputs):
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs)
        return self.norm(x)
    
class Transformer(nn.Module):
    def __init__(self, src_emb, trg_emb, N, heads,label_dim,d_ff):
        super().__init__()
        self.encoder = Encoder(src_emb, N, heads,label_dim,d_ff)
        self.decoder = Decoder(trg_emb, N, heads,d_ff,label_dim)
        self.out = nn.Linear(trg_emb, src_emb)
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, src, trg):
        e_outputs = self.encoder(src)
        d_output = self.decoder(trg, e_outputs)
        output = self.out(d_output)
        return d_output


##Training and tesing the model

In [ ]:

bs = 2
label_dim = 8

class Dataloader(Dataset):

    def __init__(self, train, label, transform=None):
        self.train = train
        self.label = label
        self.transform = None

    def __len__(self):
        return len(self.train)

    def __getitem__(self, idx):
        if self.transform:
            sample = self.transform(sample)
        else:
            return torch.tensor(self.train[idx]),torch.tensor(self.label[idx])
            #return torch.tensor(self.train[idx:idx+bs]),torch.tensor(self.label[idx:idx+bs])


#optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum = 0.1)

loss_fn_1 = nn.CrossEntropyLoss()


print('The code can run properly')


def get_accuracy(output, trg):
    length_of_seq = output.shape[0]
    acc = (torch.sum(torch.argmax(output,dim=1) == torch.argmax(trg,dim=1))).item()/length_of_seq
    return acc

    

loader = Dataloader(train,label)
val_loader = Dataloader(val,val_label)

n_head =[1,2]
n_enoderlayer = [2,4,6]
d_feedforward = [256,512,1024]
list_loss1 = []
list_loss2 = []

list_a1 =[]
list_a2 =[]

list_l = [list_loss1,list_loss2]
list_a = [list_a1,list_a2]

for i in range(2):

    h = n_head[i]
    l = list_l[i]
    a = list_a[i]
    
    model = Transformer(22,8,4,h,label_dim,256)
    model = model.double()
    model = model.to('cuda')
    epochs = 25
    ######################################################
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    with torch.autograd.set_detect_anomaly(True):
        for epoch in range(epochs):
            epoch_loss = 0
            train_acc = 0
            
            for i in range(0,500):
                loss = 0
                acc = 0
                e = np.random.randint(0,5000)
                src, trg = loader[e]
                
                src = src.to('cuda')
                trg = trg.to('cuda')
                optimizer.zero_grad()
                output = model(src,trg)
                src_mask = (src[:,21] != 1) 
                          
                masked_trg = trg[src_mask]
                masked_output = output[src_mask]
                
                #for i in range(masked_trg.shape[0]):
                #    if 1 not in masked_trg[i]:
                #        print('haha')
              
                loss = loss_fn_1(masked_output, torch.argmax(masked_trg,1))
                acc = get_accuracy(masked_output,masked_trg)
                train_acc += acc
                #loss = loss_fn_1(masked_output[1], torch.argmax(masked_trg[1],1)) + loss_fn_1(masked_output[0], torch.argmax(masked_trg[0],1))  
                loss.backward()
                optimizer.step()
                epoch_loss += float(loss)  
            l.append(epoch_loss)
            a.append(train_acc/500)
       
            
            print(f'After epoch {epoch+1}: The loss is {epoch_loss}')
            print(f'After epoch {epoch+1}: The accuracy is {train_acc/500}')
    



    # Specify a path
    PATH = "transformer.pt"

    # Save
    torch.save(model.state_dict(), PATH)

    # Load
    model = Transformer(22,8,4,h,label_dim,256)
    model = model.to('cuda')
    model = model.double()
    model.load_state_dict(torch.load(PATH))
    model.eval()

    test_loader = Dataloader(test,test_label)
    test_epoch_loss = 0
    test_epoch_acc = 0
    for i in range(0,267):
          src, trg = test_loader[i]
          src = src.to('cuda')
          trg = trg.to('cuda')
          src_mask = (src[:,21] != 1)            
          masked_src = src[src_mask]
          masked_trg = trg[src_mask]
          optimizer.zero_grad()
          output = model(masked_src,masked_trg)
          test_loss = loss_fn_1(output, torch.argmax(masked_trg,1))
          test_acc = get_accuracy(output,masked_trg)
          test_epoch_acc += test_acc
          test_epoch_loss += test_loss
    print(f'The loss is {test_epoch_loss}')
    print(f'The accuracy is {test_epoch_acc/267}')

print(list_l[0],list_l[1],list_a[0],list_a[1])
list_epoch = np.arange(epochs)
plt.plot(list_l[0],list_epoch,label='1 head')
plt.plot(list_l[1],list_epoch,label='2 heads')
plt.ylabel('Loss')
plt.xlabel('Number of Epoches')
plt.legend()
plt.show()
        
        
plt.plot(list_a[0],list_epoch,label='1 head')
plt.plot(list_a[1],list_epoch,label='2 heads')
plt.ylabel('Loss')
plt.xlabel('Number of Epoches')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
      


In [ ]:
print(list_a[0])
print(list_a[1])
print(list_l[0])
print(list_l[1])

In [ ]:
n_enoderlayer = [2,4,6]
d_feedforward = [256,512,1024]
list_loss1 = []
list_loss2 = []
list_loss3 = []
list_a1 =[]
list_a2 =[]
list_a3 =[]
list_l = [list_loss1,list_loss2,list_loss3]
list_a = [list_a1,list_a2,list_a3]
 

for i in range(3):


    l = list_l[i]
    a = list_a[i]
    
    model = Transformer(22,8,h,2,label_dim,256)
    model = model.double()
    model = model.to('cuda')
    epochs = 20
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    with torch.autograd.set_detect_anomaly(True):
        for epoch in range(epochs):
            epoch_loss = 0
            train_acc = 0
            
            for i in range(0,500):
                loss = 0
                acc = 0
                e = np.random.randint(0,5000)
                src, trg = loader[e]
                
                src = src.to('cuda')
                trg = trg.to('cuda')
                optimizer.zero_grad()
                output = model(src,trg)
                src_mask = (src[:,21] != 1) 
                          
                masked_trg = trg[src_mask]
                masked_output = output[src_mask]
                
                #for i in range(masked_trg.shape[0]):
                #    if 1 not in masked_trg[i]:
                #        print('haha')
              
                loss = loss_fn_1(masked_output, torch.argmax(masked_trg,1))
                acc = get_accuracy(masked_output,masked_trg)
                train_acc += acc
                #loss = loss_fn_1(masked_output[1], torch.argmax(masked_trg[1],1)) + loss_fn_1(masked_output[0], torch.argmax(masked_trg[0],1))  
                loss.backward()
                optimizer.step()
                epoch_loss += float(loss)  
            l.append(epoch_loss)
            a.append(train_acc/500)
       
            
            print(f'After epoch {epoch+1}: The loss is {epoch_loss}')
            print(f'After epoch {epoch+1}: The accuracy is {train_acc/500}')
    



    # Specify a path
    PATH = "transformer.pt"

    # Save
    torch.save(model.state_dict(), PATH)

    # Load
    model = Transformer(22,8,h,2,label_dim,256)
    model = model.to('cuda')
    model = model.double()
    model.load_state_dict(torch.load(PATH))
    model.eval()

    test_loader = Dataloader(test,test_label)
    test_epoch_loss = 0
    test_epoch_acc = 0
    for i in range(0,267):
          src, trg = test_loader[i]
          src = src.to('cuda')
          trg = trg.to('cuda')
          src_mask = (src[:,21] != 1)            
          masked_src = src[src_mask]
          masked_trg = trg[src_mask]
          optimizer.zero_grad()
          output = model(masked_src,masked_trg)
          test_loss = loss_fn_1(output, torch.argmax(masked_trg,1))
          test_acc = get_accuracy(output,masked_trg)
          test_epoch_acc += test_acc
          test_epoch_loss += test_loss
    print(f'The loss is {test_epoch_loss}')
    print(f'The accuracy is {test_epoch_acc/267}')

list_epoch = np.arange(epochs)
plt.plot(list_l[0],list_epoch,label='2 layers')
plt.plot(list_l[1],list_epoch,label='4 layers')
plt.plot(list_l[2],list_epoch,label='6 layers')
plt.ylabel('Loss')
plt.xlabel('Number of Epoches')
plt.legend()
plt.show()
        
        
plt.plot(list_a[0],list_epoch,label='2 layers')
plt.plot(list_a[1],list_epoch,label='4 layers')
plt.plot(list_a[2],list_epoch,label='6 layers')

plt.xlabel('Number of Epoches')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
print(list_a[0])
print(list_a[1])
print(list_a[2])
print(list_l[0])
print(list_l[1])
print(list_l[2])

In [ ]:
d_feedforward = [256,512,1024]
list_loss1 = []
list_loss2 = []
list_loss3 = []
list_a1 =[]
list_a2 =[]
list_a3 =[]
list_l = [list_loss1,list_loss2,list_loss3]
list_a = [list_a1,list_a2,list_a3]
 

for i in range(3):


    l = list_l[i]
    a = list_a[i]
    
    model = Transformer(22,8,4,2,label_dim,h)
    model = model.double()
    model = model.to('cuda')
    epochs = 30
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    with torch.autograd.set_detect_anomaly(True):
        for epoch in range(epochs):
            epoch_loss = 0
            train_acc = 0
            
            for i in range(0,1000):
                loss = 0
                acc = 0
                e = np.random.randint(0,5000)
                src, trg = loader[e]
                
                src = src.to('cuda')
                trg = trg.to('cuda')
                optimizer.zero_grad()
                output = model(src,trg)
                src_mask = (src[:,21] != 1) 
                          
                masked_trg = trg[src_mask]
                masked_output = output[src_mask]
                
                #for i in range(masked_trg.shape[0]):
                #    if 1 not in masked_trg[i]:
                #        print('haha')
              
                loss = loss_fn_1(masked_output, torch.argmax(masked_trg,1))
                acc = get_accuracy(masked_output,masked_trg)
                train_acc += acc
                #loss = loss_fn_1(masked_output[1], torch.argmax(masked_trg[1],1)) + loss_fn_1(masked_output[0], torch.argmax(masked_trg[0],1))  
                loss.backward()
                optimizer.step()
                epoch_loss += float(loss)  
            l.append(epoch_loss)
            a.append(train_acc/1000)
       
            
            print(f'After epoch {epoch+1}: The loss is {epoch_loss}')
            print(f'After epoch {epoch+1}: The accuracy is {train_acc/1000}')
    



    # Specify a path
    PATH = "transformer.pt"

    # Save
    torch.save(model.state_dict(), PATH)

    # Load
    model = Transformer(22,8,4,2,label_dim,h)
    model = model.to('cuda')
    model = model.double()
    model.load_state_dict(torch.load(PATH))
    model.eval()

    test_loader = Dataloader(test,test_label)
    test_epoch_loss = 0
    test_epoch_acc = 0
    for i in range(0,267):
          src, trg = test_loader[i]
          src = src.to('cuda')
          trg = trg.to('cuda')
          src_mask = (src[:,21] != 1)            
          masked_src = src[src_mask]
          masked_trg = trg[src_mask]
          optimizer.zero_grad()
          output = model(masked_src,masked_trg)
          test_loss = loss_fn_1(output, torch.argmax(masked_trg,1))
          test_acc = get_accuracy(output,masked_trg)
          test_epoch_acc += test_acc
          test_epoch_loss += test_loss
    print(f'The loss is {test_epoch_loss}')
    print(f'The accuracy is {test_epoch_acc/267}')

list_epoch = np.arange(epochs)
plt.plot(list_l[0],list_epoch,label='256')
plt.plot(list_l[1],list_epoch,label='512')
plt.plot(list_l[2],list_epoch,label='1024')
plt.ylabel('Loss')
plt.xlabel('Number of Epoches')
plt.legend()
plt.show()
        
        
plt.plot(list_a[0],list_epoch,label='256')
plt.plot(list_a[1],list_epoch,label='512')
plt.plot(list_a[2],list_epoch,label='1024')
plt.xlabel('Number of Epoches')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:

print(list_a[0])
print(list_a[1])
print(list_a[2])
print(list_l[0])
print(list_l[1])
print(list_l[2])

## Including the Solvent Accessibility 

In [ ]:
bs = 2
label_dim = 8


#for i in range(267):
                #    val_src, val_trg = val_loader[i]
                #    val_src = val_src.to('cuda')
                #    val_trg = val_trg.to('cuda')
                #    val_output = model(val_src,val_trg)
                #    val_src_mask = (val_src[:,21] != 1) 
                #    
                #    val_masked_trg = val_trg[val_src_mask]
                #    masked_output = val_output[val_src_mask]
                #    
                #    val_acc = get_accuracy(masked_output,val_masked_trg)
                #    total_acc.append(val_acc)
                #    print(sum(total_acc)/len(total_acc))
model = Transformer(22,8,1,1,label_dim,128)
model = model.double()
model = model.to('cuda')
epochs = 30
l1 = []
a1 = []
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(epochs):
        epoch_loss = 0
        train_acc = 0
        
        for i in range(0,2000):
            loss = 0
            acc = 0
            e = np.random.randint(0,5000)
            src, trg = loader[e]
            src = src[:,8]
            src1 = src[:,8:13]
            src = src.to('cuda')
            trg = trg.to('cuda')
            optimizer.zero_grad()
            output = model(src,trg)
            src_mask = (src[:,21] != 1) 
                      
            masked_trg = trg[src_mask]
            masked_output = output[src_mask]
            
            #for i in range(masked_trg.shape[0]):
            #    if 1 not in masked_trg[i]:
            #        print('haha')
            
            loss = loss_fn_1(masked_output, torch.argmax(masked_trg,1))
            acc = get_accuracy(masked_output,masked_trg)
            train_acc += acc
            #loss = loss_fn_1(masked_output[1], torch.argmax(masked_trg[1],1)) + loss_fn_1(masked_output[0], torch.argmax(masked_trg[0],1))  
            loss.backward()
            optimizer.step()
            epoch_loss += float(loss)  
        l1.append(epoch_loss)
        a1.append(train_acc/500)
    
        
        print(f'After epoch {epoch+1}: The loss is {epoch_loss}')
        print(f'After epoch {epoch+1}: The accuracy is {train_acc/2000}')




# Specify a path
PATH = "transformer.pt"

# Save
torch.save(model.state_dict(), PATH)

# Load
model = Transformer(22,8,1,1,label_dim,128)
model = model.to('cuda')
model = model.double()
model.load_state_dict(torch.load(PATH))
model.eval()

test_loader = Dataloader(test,test_label)
test_epoch_loss = 0
test_epoch_acc = 0
for i in range(0,267):
      src, trg = test_loader[i]
      src = src.to('cuda')
      trg = trg.to('cuda')
      src_mask = (src[:,21] != 1)            
      masked_src = src[src_mask]
      masked_trg = trg[src_mask]
      optimizer.zero_grad()
      output = model(masked_src,masked_trg)
      test_loss = loss_fn_1(output, torch.argmax(masked_trg,1))
      test_acc = get_accuracy(output,masked_trg)
      test_epoch_acc += test_acc
      test_epoch_loss += test_loss
print(f'The loss is {test_epoch_loss}')
print(f'The accuracy is {test_epoch_acc/267}')



In [ ]:
print([i * 2.5 for i in a1])
print(l1)


In [ ]:
loader1 = Dataloader(train,ss_label)
model = Transformer(22,13,1,1,13,128)
model = model.double()
model = model.to('cuda')
epochs = 30
l2 = []
a2 = []
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(epochs):
        epoch_loss = 0
        train_acc = 0
        
        for i in range(0,2000):
            loss = 0
            acc = 0
            e = np.random.randint(0,5000)
            src, trg = loader1[e]
            
            src = src.to('cuda')
            trg = trg.to('cuda')
            optimizer.zero_grad()
            output = model(src,trg)
            src_mask = (src[:,21] != 1) 
                      
            masked_trg = trg[src_mask]
            masked_output = output[src_mask]
            
            #for i in range(masked_trg.shape[0]):
            #    if 1 not in masked_trg[i]:
            #        print('haha')
          
            loss = loss_fn_1(masked_output, torch.argmax(masked_trg,1))
            acc = get_accuracy(masked_output,masked_trg)
            train_acc += acc
            #loss = loss_fn_1(masked_output[1], torch.argmax(masked_trg[1],1)) + loss_fn_1(masked_output[0], torch.argmax(masked_trg[0],1))  
            loss.backward()
            optimizer.step()
            epoch_loss += float(loss)  
        l2.append(epoch_loss)
        a2.append(train_acc/2000)
    
        
        print(f'After epoch {epoch+1}: The loss is {epoch_loss}')
        print(f'After epoch {epoch+1}: The accuracy is {train_acc/2000}')




# Specify a path
PATH = "transformer.pt"

# Save
torch.save(model.state_dict(), PATH)

# Load
model = Transformer(22,13,1,1,13,128)
model = model.to('cuda')
model = model.double()
model.load_state_dict(torch.load(PATH))
model.eval()

test_loader1 = Dataloader(test,ss_test_label)
test_epoch_loss = 0
test_epoch_acc = 0
for i in range(0,267):
      src, trg = test_loader1[i]
      src = src.to('cuda')
      trg = trg.to('cuda')
      src_mask = (src[:,21] != 1)            
      masked_src = src[src_mask]
      masked_trg = trg[src_mask]
      optimizer.zero_grad()
      output = model(masked_src,masked_trg)
      test_loss = loss_fn_1(output, torch.argmax(masked_trg,1))
      test_acc = get_accuracy(output,masked_trg)
      test_epoch_acc += test_acc
      test_epoch_loss += test_loss
print(f'The loss is {test_epoch_loss}')
print(f'The accuracy is {test_epoch_acc/267}')

list_epoch = np.arange(epochs)
plt.plot(l1,list_epoch,label='eight state')
plt.plot(l2,list_epoch,label='three state')

plt.ylabel('Loss')
plt.xlabel('Number of Epoches')
plt.legend()
plt.show()
        
        
plt.plot(a1,list_epoch,label='Without Solvent Accessibility')
plt.plot(a2,list_epoch,label='With Solvent Accessibility')


plt.ylabel('Loss')
plt.xlabel('Number of Epoches')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

##Three State Label

In [ ]:
model = Transformer(22,8,1,1,label_dim,128)
model = model.double()
model = model.to('cuda')
epochs = 30
l1 = []
a1 = []
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(epochs):
        epoch_loss = 0
        train_acc = 0
        
        for i in range(0,2000):
            loss = 0
            acc = 0
            e = np.random.randint(0,5000)
            src, trg = loader[e]
            
            src = src.to('cuda')
            trg = trg.to('cuda')
            optimizer.zero_grad()
            output = model(src,trg)
            src_mask = (src[:,21] != 1) 
                      
            masked_trg = trg[src_mask]
            masked_output = output[src_mask]
            
            #for i in range(masked_trg.shape[0]):
            #    if 1 not in masked_trg[i]:
            #        print('haha')
          
            loss = loss_fn_1(masked_output, torch.argmax(masked_trg,1))
            acc = get_accuracy(masked_output,masked_trg)
            train_acc += acc
            #loss = loss_fn_1(masked_output[1], torch.argmax(masked_trg[1],1)) + loss_fn_1(masked_output[0], torch.argmax(masked_trg[0],1))  
            loss.backward()
            optimizer.step()
            epoch_loss += float(loss)  
        l1.append(epoch_loss)
        a1.append(train_acc/2000)
    
        
        print(f'After epoch {epoch+1}: The loss is {epoch_loss}')
        print(f'After epoch {epoch+1}: The accuracy is {train_acc/2000}')




# Specify a path
PATH = "transformer.pt"

# Save
torch.save(model.state_dict(), PATH)

# Load
model = Transformer(22,8,1,1,label_dim,128)
model = model.to('cuda')
model = model.double()
model.load_state_dict(torch.load(PATH))
model.eval()

test_loader = Dataloader(test,test_label)
test_epoch_loss = 0
test_epoch_acc = 0
for i in range(0,267):
      src, trg = test_loader[i]
      src = src.to('cuda')
      trg = trg.to('cuda')
      src_mask = (src[:,21] != 1)            
      masked_src = src[src_mask]
      masked_trg = trg[src_mask]
      optimizer.zero_grad()
      output = model(masked_src,masked_trg)
      test_loss = loss_fn_1(output, torch.argmax(masked_trg,1))
      test_acc = get_accuracy(output,masked_trg)
      test_epoch_acc += test_acc
      test_epoch_loss += test_loss
print(f'The loss is {test_epoch_loss}')
print(f'The accuracy is {test_epoch_acc/267}')


In [ ]:
loader1 = Dataloader(train,label_3)
model = Transformer(22,3,1,1,3,128)
model = model.double()
model = model.to('cuda')
epochs = 30
l2 = []
a2 = []
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(epochs):
        epoch_loss = 0
        train_acc = 0
        
        for i in range(0,2000):
            loss = 0
            acc = 0
            e = np.random.randint(0,5000)
            src, trg = loader1[e]
            
            src = src.to('cuda')
            trg = trg.to('cuda')
            optimizer.zero_grad()
            output = model(src,trg)
            src_mask = (src[:,21] != 1) 
                      
            masked_trg = trg[src_mask]
            masked_output = output[src_mask]
            
            #for i in range(masked_trg.shape[0]):
            #    if 1 not in masked_trg[i]:
            #        print('haha')
          
            loss = loss_fn_1(masked_output, torch.argmax(masked_trg,1))
            acc = get_accuracy(masked_output,masked_trg)
            train_acc += acc
            #loss = loss_fn_1(masked_output[1], torch.argmax(masked_trg[1],1)) + loss_fn_1(masked_output[0], torch.argmax(masked_trg[0],1))  
            loss.backward()
            optimizer.step()
            epoch_loss += float(loss)  
        l2.append(epoch_loss)
        a2.append(train_acc/200)
    
        
        print(f'After epoch {epoch+1}: The loss is {epoch_loss}')
        print(f'After epoch {epoch+1}: The accuracy is {train_acc/2000}')




# Specify a path
PATH = "transformer.pt"

# Save
torch.save(model.state_dict(), PATH)

# Load
model = Transformer(22,3,1,1,3,128)
model = model.to('cuda')
model = model.double()
model.load_state_dict(torch.load(PATH))
model.eval()

test_loader1 = Dataloader(test,test_label_3)
test_epoch_loss = 0
test_epoch_acc = 0
for i in range(0,267):
      src, trg = test_loader1[i]
      src = src.to('cuda')
      trg = trg.to('cuda')
      src_mask = (src[:,21] != 1)            
      masked_src = src[src_mask]
      masked_trg = trg[src_mask]
      optimizer.zero_grad()
      output = model(masked_src,masked_trg)
      test_loss = loss_fn_1(output, torch.argmax(masked_trg,1))
      test_acc = get_accuracy(output,masked_trg)
      test_epoch_acc += test_acc
      test_epoch_loss += test_loss
print(f'The loss is {test_epoch_loss}')
print(f'The accuracy is {test_epoch_acc/267}')



In [ ]:
print(a2)
print(l2)

In [ ]:
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################

##Validate the model